In [9]:
from datasets import load_dataset, concatenate_datasets

agg = load_dataset("billingsmoore/Aggregated-bo-en", split='train')

op_train = load_dataset("openpecha/tagged_cleaned_MT_v1.0.3", split='train')
op_test = load_dataset("openpecha/tagged_cleaned_MT_v1.0.3", split='test')
op = concatenate_datasets([op_train, op_test])

op = op.rename_column('Source', 'bo')
op = op.rename_column('Target', 'en')

In [2]:
agg

Dataset({
    features: ['bo', 'en', 'source_file', 'source', 'topic'],
    num_rows: 885100
})

In [ ]:
bud_agg = agg.filter(lambda example: example['source'] in ['84000', 'LotsawaHouse'])
set(bud_agg['source'])

Filter:   0%|          | 0/885100 [00:00<?, ? examples/s]

In [10]:
op

Dataset({
    features: ['bo', 'en', 'File_Name', 'Machine Aligned', '__index_level_0__', 'Tag'],
    num_rows: 1438258
})

In [11]:
op[0]['Tag']

{'Buddhist': True,
 'LH labels': ['Akṣobhya', 'Longsal Dorje Nyingpo'],
 'Topic': 'Prophecies, Rituals'}

In [13]:
bud_op = op.filter(lambda example: example['Tag']['Buddhist'])
bud_op


Filter:   0%|          | 0/1438258 [00:00<?, ? examples/s]

Dataset({
    features: ['bo', 'en', 'File_Name', 'Machine Aligned', '__index_level_0__', 'Tag'],
    num_rows: 866512
})

In [20]:
conc = concatenate_datasets([bud_agg, bud_op]).remove_columns(['source_file', 'source', 'topic', 'File_Name', 'Machine Aligned', '__index_level_0__', 'Tag'])
conc = conc.shuffle(seed=42)
conc

Dataset({
    features: ['bo', 'en'],
    num_rows: 1029132
})

In [24]:
df = conc.to_pandas()
df.head()

,bo,en
0,སྤྱིར་ཡང་འདི་ལ་ཛ་ག་ནཱ་ཐ་སྟེ་འགྲོ་བའི་མགོན་པོར་...,"There are those who say that, in general, Jaga..."
1,དེ་ཅིའི་ཕྱིར་ཞེ་ན། ཀཽ་ཤི་ཀ་དེ་ནི་འདི་ལྟར་དོན་ད...,"If you ask why, Kauśika, it is because the bou..."
2,དེས་ན། མིའི་སྤྱི་ཚོགས་ཁྲོད་དུ་བྱམས་པ་དང་སྙིང་ར...,"Thus in human society. love, compassion, and k..."
3,སྙིང་པོའི་མདོ་བཅུ་དང་། རྒྱུད་བླ་མ་སོགས་རྒྱུ་ཡི...,By only this there is no entailment because th...
4,གཏེར་རྡོལ་ཁའི་ཁ་མ་བཀབ།།,Don’t close the lid of the treasure.


In [25]:
len(df)

1029132

In [26]:
df = df.drop_duplicates(subset='en')
len(df)

1018441

In [27]:
df = df.drop_duplicates(subset='bo')
len(df)

1011417

In [31]:
from datasets import Dataset

conc = Dataset.from_pandas(df[['bo', 'en']]).remove_columns(['__index_level_0__'])
conc

Dataset({
    features: ['bo', 'en'],
    num_rows: 1011417
})

In [32]:
from datasets import DatasetDict

test = conc.select(range(100_000))
dev = conc.select(range(100_000, 150_000))
train = conc.select(range(150_000, len(conc)))

ds = DatasetDict()

ds['train'] = train
ds['dev'] = dev
ds['test'] = test

ds

DatasetDict({
    train: Dataset({
        features: ['bo', 'en'],
        num_rows: 861417
    })
    dev: Dataset({
        features: ['bo', 'en'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['bo', 'en'],
        num_rows: 100000
    })
})

In [33]:
ds.save_to_disk('raw-ds')

Saving the dataset (0/2 shards):   0%|          | 0/861417 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]